In [15]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import numpy as np
import os

# YOLO 모델 로드
model = YOLO(r'C:\KDT_SF6\비전ai\best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),   # yaozhe - 빨강
    1: (0, 255, 0),   # waist folding - 초록
    2: (0, 0, 255),   # punching hole - 파랑
    # 나머지 클래스 색상 추가
}

# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (상단에 위치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, columnspan=2, pady=10, sticky="n")

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

# 프레임의 크기를 고정
summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(*summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 탐지 결과를 저장할 딕셔너리
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_list, file_index
    if file_list and file_index < len(file_list):
        file_path = os.path.join(selected_folder, file_list[file_index])
        predict_image(file_path)
        file_index += 1

    # 더 이상 파일이 없으면 종료
    if file_index >= len(file_list):
        result_label.config(text="모든 파일을 처리하였습니다.")
        return

    # 2초마다 이 함수를 다시 호출
    window.after(2000, check_for_new_image)

# 폴더 선택 및 이미지 처리 함수
def select_folder():
    global file_list, file_index, selected_folder
    selected_folder = filedialog.askdirectory()  # 폴더 선택 다이얼로그
    if selected_folder:
        file_list = [f for f in os.listdir(selected_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        file_index = 0
        if file_list:
            check_for_new_image()

# 개별 파일 선택 및 예측 수행 함수
def upload_and_predict():
    file_path = filedialog.askopenfilename(title="이미지 파일 선택", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        predict_image(file_path)

def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 각 클래스별 탐지된 개수를 저장할 임시 딕셔너리
        current_detection_count = {key: 0 for key in custom_names.values()}

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy
            cls = r.boxes.cls
            conf = r.boxes.conf
            cls_dict = r.names

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box
                x1_int, y1_int = int(x1.item()), int(y1.item())
                x2_int, y2_int = int(x2.item()), int(y2.item())
                cls_name = cls_dict[int(cls_number.item())]
                conf_number = float(conf_value.item())

                # 탐지된 객체 수 업데이트
                current_detection_count[cls_name] += 1
                detection_summary[cls_name] += 1

                # 총합 계산
                total_sums[cls_name] += 1

                final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                    "cls_name": cls_name, "conf": conf_number})

        # 테이블에 결과 추가
        new_data = (
            os.path.basename(file_path),
            current_detection_count['waist folding'],
            current_detection_count['punching hole'],
            current_detection_count['welding line'],
            current_detection_count['crescent gap'],
            current_detection_count['water spot'],
            current_detection_count['oil spot'],
            current_detection_count['silk spot'],
            current_detection_count['inclusion'],
            current_detection_count['rolled pit'],
            current_detection_count['crease']
        )

        tree.insert("", tk.END, values=new_data)
        # 탐지된 객체 수와 각 객체의 정보를 result_label에 업데이트
        detection_summary_text = f"탐지된 오류 수: {len(final_boxes)}\n\n"
        detection_summary_text += "\n".join(detection_details)  # 탐지된 객체 정보를 줄바꿈으로 나열
        
        result_label.config(text=detection_summary_text, font=("Helvetica", 16))  # 폰트 크기 16으로 설정

        # 총합 업데이트
        update_summary()

        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택

            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 2)
            image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # 탐지된 객체 수를 출력하는 부분
        result_label.config(text=f"탐지된 객체 수: {len(final_boxes)}")

        # 이미지 표시
        show_image_from_array(image)

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")

# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

result_label = tk.Label(window, text="")
result_label.grid(pady=1)

# 폴더 선택 버튼
folder_button = tk.Button(window, text="폴더 선택 및 분석", command=select_folder)
folder_button.grid(row=4, column=0, pady=10)

# 파일 업로드 버튼 추가
upload_button = tk.Button(window, text="파일 업로드 및 분석", command=upload_and_predict)
upload_button.grid(row=4, column=1, pady=10)

window.mainloop()



image 1/1 C:\KDT_SF6\ai\  \img_01_424799300_01133.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 31.9ms
Speed: 1.0ms preprocess, 31.9ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)


In [16]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import numpy as np
import os

# YOLO 모델 로드
model = YOLO(r'C:\KDT_SF6\비전ai\best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),   # yaozhe - 빨강
    1: (0, 255, 0),   # waist folding - 초록
    2: (0, 0, 255),   # punching hole - 파랑
    # 나머지 클래스 색상 추가
}

# 이미지 저장 경로 설정
save_path = r'C:\KDT_SF6\비전ai'  # 저장할 폴더 경로

# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# 탐지 결과를 저장할 딕셔너리 (초기값 0)
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (상단에 위치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, columnspan=2, pady=10, sticky="n")

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(*summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_list, file_index
    if file_list and file_index < len(file_list):
        file_path = os.path.join(selected_folder, file_list[file_index])
        predict_image(file_path)
        file_index += 1

    # 더 이상 파일이 없으면 종료
    if file_index >= len(file_list):
        result_label.config(text="모든 파일을 처리하였습니다.")
        return

    # 2초마다 이 함수를 다시 호출
    window.after(2000, check_for_new_image)

# 폴더 선택 및 이미지 처리 함수
def select_folder():
    global file_list, file_index, selected_folder
    selected_folder = filedialog.askdirectory()  # 폴더 선택 다이얼로그
    if selected_folder:
        file_list = [f for f in os.listdir(selected_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        file_index = 0
        if file_list:
            check_for_new_image()

# 개별 파일 선택 및 예측 수행 함수
def upload_and_predict():
    file_path = filedialog.askopenfilename(title="이미지 파일 선택", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        predict_image(file_path)

def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)  # 이미지 로드
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV 이미지를 RGB로 변환
        h, w, c = image.shape  # 이미지 크기 저장 (높이, 너비, 채널)

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy  # 바운딩 박스 좌표
            cls = r.boxes.cls  # 클래스 번호
            conf = r.boxes.conf  # 신뢰도
            cls_dict = r.names  # 클래스 이름

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box  # 바운딩 박스 좌표
                x1_int, y1_int = int(x1.item()), int(y1.item())  # 좌상단 좌표
                x2_int, y2_int = int(x2.item()), int(y2.item())  # 우하단 좌표
                cls_name = cls_dict[int(cls_number.item())]  # 클래스 이름
                conf_number = float(conf_value.item())  # 신뢰도

                # 바운딩 박스를 최종 리스트에 추가
                final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                    "cls_name": cls_name, "conf": conf_number})

        # 이미지 중심 좌표 계산
        center_image_x = w // 2
        center_image_y = h // 2

        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"  # 클래스 이름과 신뢰도

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택

            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 3)

            # 바운딩 박스의 중앙 좌표 계산
            center_box_x = int((x1_int + x2_int) / 2)
            center_box_y = int((y1_int + y2_int) / 2)

            # 이미지 중심과 바운딩 박스 중심 사이에서 바운딩 박스에 30% 더 가까운 지점 계산
            mid_x = int(center_image_x + 0.8 * (center_box_x - center_image_x))
            mid_y = int(center_image_y + 0.8 * (center_box_y - center_image_y))

            # 텍스트가 이미지 중심과 바운딩 박스 사이에서 30% 더 바운딩 박스에 가깝게 위치하도록 조정
            image = cv2.putText(image, label, (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # 탐지된 객체 수를 출력하는 부분
        result_label.config(text=f"탐지된 객체 수: {len(final_boxes)}")

        # 이미지 표시
        show_image_from_array(image)

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")
        
# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

result_label = tk.Label(window, text="")
result_label.grid(pady=1)

# 폴더 선택 버튼
folder_button = tk.Button(window, text="폴더 선택 및 분석", command=select_folder)
folder_button.grid(row=4, column=0, pady=10)

# 파일 업로드 버튼 추가
upload_button = tk.Button(window, text="파일 업로드 및 분석", command=upload_and_predict)
upload_button.grid(row=4, column=1, pady=10)

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

window.mainloop()


In [13]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import numpy as np
import os

# YOLO 모델 로드
model = YOLO(r'C:\KDT_SF6\비전ai\best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),      # yaozhe - 빨강
    1: (0, 255, 0),      # waist folding - 초록
    2: (0, 0, 255),      # punching hole - 파랑
    3: (255, 255, 0),    # welding line - 노랑
    4: (255, 0, 255),    # crescent gap - 마젠타
    5: (0, 255, 255),    # water spot - 청록
    6: (128, 0, 0),      # oil spot - 어두운 빨강
    7: (128, 128, 0),    # silk spot - 올리브
    8: (0, 128, 0),      # inclusion - 어두운 초록
    9: (0, 128, 128),    # rolled pit - 청록
    10: (0, 0, 128),     # crease - 어두운 파랑
    11: (128, 0, 128)    # d - 보라
}


# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (상단에 위치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, columnspan=2, pady=10, sticky="n")

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

# 프레임의 크기를 고정
# summary_tree_frame.grid_propagate(False)

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(*summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 파일을 가져올 폴더 경로
image_folder = r'C:\KDT_SF6\비전ai\새 폴더'
file_index = 0

# 탐지 결과를 저장할 딕셔너리
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area
    if union_area == 0:
        return 0
    return inter_area / union_area

def adjust_label_position(x1, y1, x2, y2, image_w, image_h, label_size):
    """
    바운딩 박스와 이미지 크기를 기반으로 텍스트 위치를 조정합니다.
    
    x1, y1: 바운딩 박스의 좌상단 좌표
    x2, y2: 바운딩 박스의 우하단 좌표
    image_w, image_h: 이미지의 너비와 높이
    label_size: 텍스트의 세로 크기 (폰트 크기 등에 기반)
    
    return: (x, y) - 텍스트가 그려질 좌표
    """
    # 바운딩 박스의 중간 좌표 계산
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2

    # 기본적으로 텍스트는 바운딩 박스 위에 배치 (약간의 여유 공간 확보)
    label_x = x1
    label_y = y1 - 10

    # 텍스트가 이미지 상단 경계를 벗어나는 경우 (텍스트가 위쪽에 놓일 수 없는 경우)
    if label_y < 0:
        label_y = y2 + label_size  # 텍스트를 바운딩 박스 아래로 이동

    # 텍스트가 좌측 경계를 벗어나는 경우 (너무 왼쪽)
    if label_x < 0:
        label_x = 10  # 텍스트를 이미지 왼쪽에서 약간 떨어뜨려 표시

    # 텍스트가 이미지의 우측 경계를 벗어나는 경우 (너무 오른쪽)
    if label_x + (x2 - x1) > image_w:
        label_x = image_w - (x2 - x1) - 10  # 텍스트를 우측 경계에서 떨어지게

    return label_x, label_y


# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_index
    # 폴더에서 모든 파일 가져오기
    file_list = sorted(
        (f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))),
        key=lambda x: os.path.getmtime(os.path.join(image_folder, x))
    )

    # 파일이 있는 경우에만 처리
    if file_list:
        if file_index < len(file_list):
            file_path = os.path.join(image_folder, file_list[file_index])
            predict_image(file_path)  # 파일 예측 수행
            file_index += 1

        if file_index >= len(file_list):
            print("모든 파일을 처리하였습니다.")
            return  # 더 이상 반복하지 않고 종료

    # 2초마다 이 함수를 다시 호출
    window.after(2000, check_for_new_image)

# 파일을 직접 업로드해서 분석하는 함수
def upload_and_predict():
    file_path = filedialog.askopenfilename(title="이미지 파일 선택", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        predict_image(file_path)

def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, c = image.shape

        # 탐지된 객체 정보를 담을 리스트 생성
        detection_details = []

        # 각 클래스별 탐지된 개수를 저장할 임시 딕셔너리
        current_detection_count = {key: 0 for key in custom_names.values()}

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy
            cls = r.boxes.cls
            conf = r.boxes.conf
            cls_dict = r.names

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box
                x1_int, y1_int = int(x1.item()), int(y1.item())
                x2_int, y2_int = int(x2.item()), int(y2.item())
                cls_name = cls_dict[int(cls_number.item())]
                conf_number = float(conf_value.item())
                conf_percent = float(conf_value.item()) * 100  # 신뢰도를 퍼센트로 변환

                # 탐지된 객체 수 업데이트
                current_detection_count[cls_name] += 1
                detection_summary[cls_name] += 1

                # 총합 계산
                total_sums[cls_name] += 1

                # 탐지된 객체 정보 추가
                label = f"{cls_name} ({conf_number:.2f})"

                label2 = f"{cls_name} {conf_percent:.2f}%"  # 퍼센트 형식으로 변경
                detection_details.append(label2) 
                # 중복 박스 필터링 (IoU 계산)
                should_keep = True
                for kept_box in final_boxes:
                    iou = calculate_iou((x1_int, y1_int, x2_int, y2_int), kept_box["coords"])
                    if iou > 0.3:  # 만약 IoU가 0.3 이상이면 중복된다고 판단
                        should_keep = False
                        break

                if should_keep:
                    final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                        "cls_name": cls_name, "conf": conf_number})

        # 테이블에 결과 추가
        new_data = (
            os.path.basename(file_path),
            current_detection_count['waist folding'],
            current_detection_count['punching hole'],
            current_detection_count['welding line'],
            current_detection_count['crescent gap'],
            current_detection_count['water spot'],
            current_detection_count['oil spot'],
            current_detection_count['silk spot'],
            current_detection_count['inclusion'],
            current_detection_count['rolled pit'],
            current_detection_count['crease']
        )

        tree.insert("", tk.END, values=new_data)
# 탐지된 객체 수와 각 객체의 정보를 result_label에 업데이트
        detection_summary_text = f"탐지된 오류 수: {len(final_boxes)}\n\n"
        detection_summary_text += "\n".join(detection_details)  # 탐지된 객체 정보를 줄바꿈으로 나열
        
        result_label.config(text=detection_summary_text, font=("Helvetica", 16))  # 폰트 크기 16으로 설정

        
        # 총합 업데이트
        update_summary()
        # 이미지 중심 좌표 계산
        center_image_x = w // 2
        center_image_y = h // 2
        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택
            """
            image: 바운딩 박스를 그릴 이미지.
            (x1_int, y1_int): 바운딩 박스의 좌상단 좌표 (x, y).
            (x2_int, y2_int): 바운딩 박스의 우하단 좌표 (x, y).
            box_color: 박스의 색상. (B, G, R) 형태로 각각 파란색(Blue), 녹색(Green), 빨간색(Red) 값을 받습니다. 예: (255, 0, 0)은 빨강.
            2: 박스 선의 두께 (픽셀 단위).
            """
            """
            image: 텍스트를 그릴 이미지.
            label: 표시할 텍스트 (여기서는 label = f"{custom_names[cls_name]} {conf_number:.2f}").
            (x1_int, y1_int - 10): 텍스트를 표시할 좌표. 텍스트의 시작 위치. (x1_int, y1_int - 10)는 좌상단 좌표 바로 위에 텍스트를 배치함.
            cv2.FONT_HERSHEY_SIMPLEX: 사용할 글꼴. OpenCV에서 지원하는 폰트 중 하나입니다. 다른 옵션으로는 FONT_HERSHEY_COMPLEX, FONT_HERSHEY_PLAIN 등이 있습니다.
            0.5: 텍스트의 크기(scale). 숫자가 클수록 텍스트가 커집니다.
            (255, 0, 0): 텍스트 색상. (B, G, R) 값으로 각각 파란색(Blue), 녹색(Green), 빨간색(Red) 값을 설정합니다. 여기서는 빨강 (255, 0, 0)을 사용.
            2: 텍스트의 두께. 숫자가 클수록 텍스트의 두께가 커집니다.
            """
            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 2)

            # 텍스트 그림자를 그리기 (검정색으로 약간 큰 텍스트 그리기)
            # image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 4)
           # 바운딩 박스의 중앙 좌표 계산
            center_box_x = int((x1_int + x2_int) / 2)
            center_box_y = int((y1_int + y2_int) / 2)

            # 이미지 중심과 바운딩 박스 중심 사이에서 바운딩 박스에 30% 더 가까운 지점 계산
            mid_x = int(center_image_x + 0.8 * (center_box_x - center_image_x))
            mid_y = int(center_image_y + 0.8 * (center_box_y - center_image_y))

            # 텍스트가 이미지 중심과 바운딩 박스 사이에서 30% 더 바운딩 박스에 가깝게 위치하도록 조정
            image = cv2.putText(image, label, (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 2)
            # 실제 텍스트 그리기 (기존 색상)
            # image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # 탐지된 객체 수를 출력하는 부분
        # result_label.config(text=f"탐지된 오류 수: {len(final_boxes)}")

        # 이미지 표시
        show_image_from_array(image)

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")


# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# 최종 탐지 결과를 출력하고 프로그램 종료
def display_detection_summary():
    final_data = [
        "Total",
        detection_summary['waist folding'],
        detection_summary['punching hole'],
        detection_summary['welding line'],
        detection_summary['crescent gap'],
        detection_summary['water spot'],
        detection_summary['oil spot'],
        detection_summary['silk spot'],
        detection_summary['inclusion'],
        detection_summary['rolled pit'],
        detection_summary['crease']
    ]

    # TreeView에 최종 요약 데이터를 추가
    tree.insert("", tk.END, values=final_data)

    # 창 닫기
    window.destroy()

# 이미지 표시를 위한 라벨 추가 (상단에 위치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, pady=10, sticky="n")  # 이미지 라벨을 첫 번째 열에 배치

# result_label을 이미지 오른쪽에 배치
result_label = tk.Label(window, text="")
result_label.grid(row=0, column=1, padx=10, pady=10, sticky="n")  # 이미지 오른쪽에 배치

# 프로그램 종료 시 탐지 요약 출력 및 종료
window.protocol("WM_DELETE_WINDOW", display_detection_summary)

# 파일 업로드 버튼 추가 (아래쪽에 위치)
upload_button = tk.Button(window, text="파일 업로드 및 분석", command=upload_and_predict)
upload_button.grid(row=4, column=0, columnspan=2, pady=10)

# 2초마다 새 이미지를 확인하게 설정
window.after(2000, check_for_new_image)

window.mainloop()



image 1/1 C:\KDT_SF6\ai\ \img_01_424799600_00001.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 38.3ms
Speed: 1.2ms preprocess, 38.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 C:\KDT_SF6\ai\ \img_02_3402618000_00002.jpg: 320x640 1 10_yaozhed, 34.5ms
Speed: 2.0ms preprocess, 34.5ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 C:\KDT_SF6\ai\ \img_02_3402618000_00007.jpg: 320x640 1 10_yaozhed, 35.0ms
Speed: 1.0ms preprocess, 35.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 C:\KDT_SF6\ai\ \img_02_3402618000_00010.jpg: 320x640 1 2_hanfeng, 36.6ms
Speed: 2.0ms preprocess, 36.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)


In [5]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import numpy as np
import os

# YOLO 모델 로드
model = YOLO(r'C:\KDT_SF6\비전ai\best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),      # yaozhe - 빨강
    1: (0, 255, 0),      # waist folding - 초록
    2: (0, 0, 255),      # punching hole - 파랑
    3: (255, 255, 0),    # welding line - 노랑
    4: (255, 0, 255),    # crescent gap - 마젠타
    5: (0, 255, 255),    # water spot - 청록
    6: (128, 0, 0),      # oil spot - 어두운 빨강
    7: (128, 128, 0),    # silk spot - 올리브
    8: (0, 128, 0),      # inclusion - 어두운 초록
    9: (0, 128, 128),    # rolled pit - 청록
    10: (0, 0, 128),     # crease - 어두운 파랑
    11: (128, 0, 128)    # d - 보라
}


# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (상단에 위치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, columnspan=2, pady=10, sticky="n")

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

# 프레임의 크기를 고정
# summary_tree_frame.grid_propagate(False)

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(*summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 파일을 가져올 폴더 경로
image_folder = r'C:\KDT_SF6\비전ai\새 폴더'
file_index = 0

# 탐지 결과를 저장할 딕셔너리
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area
    if union_area == 0:
        return 0
    return inter_area / union_area

def adjust_label_position(x1, y1, x2, y2, image_w, image_h, label_size):
    """
    바운딩 박스와 이미지 크기를 기반으로 텍스트 위치를 조정합니다.
    
    x1, y1: 바운딩 박스의 좌상단 좌표
    x2, y2: 바운딩 박스의 우하단 좌표
    image_w, image_h: 이미지의 너비와 높이
    label_size: 텍스트의 세로 크기 (폰트 크기 등에 기반)
    
    return: (x, y) - 텍스트가 그려질 좌표
    """
    # 바운딩 박스의 중간 좌표 계산
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2

    # 기본적으로 텍스트는 바운딩 박스 위에 배치 (약간의 여유 공간 확보)
    label_x = x1
    label_y = y1 - 10

    # 텍스트가 이미지 상단 경계를 벗어나는 경우 (텍스트가 위쪽에 놓일 수 없는 경우)
    if label_y < 0:
        label_y = y2 + label_size  # 텍스트를 바운딩 박스 아래로 이동

    # 텍스트가 좌측 경계를 벗어나는 경우 (너무 왼쪽)
    if label_x < 0:
        label_x = 10  # 텍스트를 이미지 왼쪽에서 약간 떨어뜨려 표시

    # 텍스트가 이미지의 우측 경계를 벗어나는 경우 (너무 오른쪽)
    if label_x + (x2 - x1) > image_w:
        label_x = image_w - (x2 - x1) - 10  # 텍스트를 우측 경계에서 떨어지게

    return label_x, label_y


# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_index
    # 폴더에서 모든 파일 가져오기
    file_list = sorted(
        (f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))),
        key=lambda x: os.path.getmtime(os.path.join(image_folder, x))
    )

    # 파일이 있는 경우에만 처리
    if file_list:
        if file_index < len(file_list):
            file_path = os.path.join(image_folder, file_list[file_index])
            predict_image(file_path)  # 파일 예측 수행
            file_index += 1

        if file_index >= len(file_list):
            print("모든 파일을 처리하였습니다.")
            return  # 더 이상 반복하지 않고 종료

    # 2초마다 이 함수를 다시 호출
    window.after(2000, check_for_new_image)

# 파일을 직접 업로드해서 분석하는 함수
def upload_and_predict():
    file_path = filedialog.askopenfilename(title="이미지 파일 선택", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        predict_image(file_path)

def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, c = image.shape

        # 탐지된 객체 정보를 담을 리스트 생성
        detection_details = []

        # 각 클래스별 탐지된 개수를 저장할 임시 딕셔너리
        current_detection_count = {key: 0 for key in custom_names.values()}

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy
            cls = r.boxes.cls
            conf = r.boxes.conf
            cls_dict = r.names

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box
                x1_int, y1_int = int(x1.item()), int(y1.item())
                x2_int, y2_int = int(x2.item()), int(y2.item())
                cls_name = cls_dict[int(cls_number.item())]
                conf_number = float(conf_value.item())
                conf_percent = float(conf_value.item()) * 100  # 신뢰도를 퍼센트로 변환

                # 탐지된 객체 수 업데이트
                current_detection_count[cls_name] += 1
                detection_summary[cls_name] += 1

                # 총합 계산
                total_sums[cls_name] += 1

                # 탐지된 객체 정보 추가
                label = f"{cls_name} ({conf_number:.2f})"

                label2 = f"{cls_name} {conf_percent:.2f}%"  # 퍼센트 형식으로 변경
                detection_details.append(label2) 
                # 중복 박스 필터링 (IoU 계산)
                should_keep = True
                for kept_box in final_boxes:
                    iou = calculate_iou((x1_int, y1_int, x2_int, y2_int), kept_box["coords"])
                    if iou > 0.3:  # 만약 IoU가 0.3 이상이면 중복된다고 판단
                        should_keep = False
                        break

                if should_keep:
                    final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                        "cls_name": cls_name, "conf": conf_number})

        # 테이블에 결과 추가
        new_data = (
            os.path.basename(file_path),
            current_detection_count['waist folding'],
            current_detection_count['punching hole'],
            current_detection_count['welding line'],
            current_detection_count['crescent gap'],
            current_detection_count['water spot'],
            current_detection_count['oil spot'],
            current_detection_count['silk spot'],
            current_detection_count['inclusion'],
            current_detection_count['rolled pit'],
            current_detection_count['crease']
        )

        tree.insert("", tk.END, values=new_data)
# 탐지된 객체 수와 각 객체의 정보를 result_label에 업데이트
        detection_summary_text = f"탐지된 오류 수: {len(final_boxes)}\n\n"
        detection_summary_text += "\n".join(detection_details)  # 탐지된 객체 정보를 줄바꿈으로 나열
        
        result_label.config(text=detection_summary_text, font=("Helvetica", 16))  # 폰트 크기 16으로 설정

        
        # 총합 업데이트
        update_summary()
        # 이미지 중심 좌표 계산
        center_image_x = w // 2
        center_image_y = h // 2
        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택
            """
            image: 바운딩 박스를 그릴 이미지.
            (x1_int, y1_int): 바운딩 박스의 좌상단 좌표 (x, y).
            (x2_int, y2_int): 바운딩 박스의 우하단 좌표 (x, y).
            box_color: 박스의 색상. (B, G, R) 형태로 각각 파란색(Blue), 녹색(Green), 빨간색(Red) 값을 받습니다. 예: (255, 0, 0)은 빨강.
            2: 박스 선의 두께 (픽셀 단위).
            """
            """
            image: 텍스트를 그릴 이미지.
            label: 표시할 텍스트 (여기서는 label = f"{custom_names[cls_name]} {conf_number:.2f}").
            (x1_int, y1_int - 10): 텍스트를 표시할 좌표. 텍스트의 시작 위치. (x1_int, y1_int - 10)는 좌상단 좌표 바로 위에 텍스트를 배치함.
            cv2.FONT_HERSHEY_SIMPLEX: 사용할 글꼴. OpenCV에서 지원하는 폰트 중 하나입니다. 다른 옵션으로는 FONT_HERSHEY_COMPLEX, FONT_HERSHEY_PLAIN 등이 있습니다.
            0.5: 텍스트의 크기(scale). 숫자가 클수록 텍스트가 커집니다.
            (255, 0, 0): 텍스트 색상. (B, G, R) 값으로 각각 파란색(Blue), 녹색(Green), 빨간색(Red) 값을 설정합니다. 여기서는 빨강 (255, 0, 0)을 사용.
            2: 텍스트의 두께. 숫자가 클수록 텍스트의 두께가 커집니다.
            """
            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 2)

            # 텍스트 그림자를 그리기 (검정색으로 약간 큰 텍스트 그리기)
            # image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 4)
           # 바운딩 박스의 중앙 좌표 계산
            center_box_x = int((x1_int + x2_int) / 2)
            center_box_y = int((y1_int + y2_int) / 2)

            # 이미지 중심과 바운딩 박스 중심 사이에서 바운딩 박스에 30% 더 가까운 지점 계산
            mid_x = int(center_image_x + 0.8 * (center_box_x - center_image_x))
            mid_y = int(center_image_y + 0.8 * (center_box_y - center_image_y))

            # 텍스트가 이미지 중심과 바운딩 박스 사이에서 30% 더 바운딩 박스에 가깝게 위치하도록 조정
            image = cv2.putText(image, label, (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 2)
            # 실제 텍스트 그리기 (기존 색상)
            # image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # 탐지된 객체 수를 출력하는 부분
        # result_label.config(text=f"탐지된 오류 수: {len(final_boxes)}")

        # 이미지 표시
        show_image_from_array(image)

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")


# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# 최종 탐지 결과를 출력하고 프로그램 종료
def display_detection_summary():
    final_data = [
        "Total",
        detection_summary['waist folding'],
        detection_summary['punching hole'],
        detection_summary['welding line'],
        detection_summary['crescent gap'],
        detection_summary['water spot'],
        detection_summary['oil spot'],
        detection_summary['silk spot'],
        detection_summary['inclusion'],
        detection_summary['rolled pit'],
        detection_summary['crease']
    ]

    # TreeView에 최종 요약 데이터를 추가
    tree.insert("", tk.END, values=final_data)

    # 창 닫기
    window.destroy()

# 이미지 표시를 위한 라벨 추가 (상단에 위치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, pady=10, sticky="n")  # 이미지 라벨을 첫 번째 열에 배치

# result_label을 이미지 오른쪽에 배치
result_label = tk.Label(window, text="")
result_label.grid(row=0, column=1, padx=10, pady=10, sticky="n")  # 이미지 오른쪽에 배치

# 프로그램 종료 시 탐지 요약 출력 및 종료
window.protocol("WM_DELETE_WINDOW", display_detection_summary)

# 파일 업로드 버튼 추가 (아래쪽에 위치)
upload_button = tk.Button(window, text="파일 업로드 및 분석", command=upload_and_predict)
upload_button.grid(row=4, column=0, columnspan=2, pady=10)

# 2초마다 새 이미지를 확인하게 설정
window.after(2000, check_for_new_image)

window.mainloop()

In [10]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import numpy as np
import os

# YOLO 모델 로드
model = YOLO(r'C:\KDT_SF6\비전ai\best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),   # yaozhe - 빨강
    1: (0, 255, 0),   # waist folding - 초록
    2: (0, 0, 255),   # punching hole - 파랑
    # 나머지 클래스 색상 추가
}

# 이미지 저장 경로 설정
save_path = r'C:\KDT_SF6\비전ai'  # 저장할 폴더 경로

# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# 탐지 결과를 저장할 딕셔너리 (초기값 0)
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (상단에 위치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, columnspan=2, pady=10, sticky="n")

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(*summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_list, file_index
    if file_list and file_index < len(file_list):
        file_path = os.path.join(selected_folder, file_list[file_index])
        predict_image(file_path)
        file_index += 1

    # 더 이상 파일이 없으면 종료
    if file_index >= len(file_list):
        result_label.config(text="모든 파일을 처리하였습니다.")
        return

    # 2초마다 이 함수를 다시 호출
    window.after(2000, check_for_new_image)

# 폴더 선택 및 이미지 처리 함수
def select_folder():
    global file_list, file_index, selected_folder
    selected_folder = filedialog.askdirectory()  # 폴더 선택 다이얼로그
    if selected_folder:
        file_list = [f for f in os.listdir(selected_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        file_index = 0
        if file_list:
            check_for_new_image()

# 개별 파일 선택 및 예측 수행 함수
def upload_and_predict():
    file_path = filedialog.askopenfilename(title="이미지 파일 선택", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        predict_image(file_path)

def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 각 클래스별 탐지된 개수를 저장할 임시 딕셔너리
        current_detection_count = {key: 0 for key in custom_names.values()}
        detection_details = []  # 탐지된 객체의 상세 정보를 저장할 리스트

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy
            cls = r.boxes.cls
            conf = r.boxes.conf
            cls_dict = r.names

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box
                x1_int, y1_int = int(x1.item()), int(y1.item())
                x2_int, y2_int = int(x2.item()), int(y2.item())
                cls_name = cls_dict[int(cls_number.item())]
                conf_number = float(conf_value.item())

                # 탐지된 객체 수 업데이트
                current_detection_count[cls_name] += 1
                detection_summary[cls_name] += 1

                # 총합 계산
                total_sums[cls_name] += 1

                # 탐지된 객체 정보 저장
                detection_details.append(f"{cls_name}: ({x1_int}, {y1_int})에서 ({x2_int}, {y2_int}) [확률: {conf_number:.2f}]")

                final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                    "cls_name": cls_name, "conf": conf_number})

        # 테이블에 결과 추가
        new_data = (
            os.path.basename(file_path),
            current_detection_count['waist folding'],
            current_detection_count['punching hole'],
            current_detection_count['welding line'],
            current_detection_count['crescent gap'],
            current_detection_count['water spot'],
            current_detection_count['oil spot'],
            current_detection_count['silk spot'],
            current_detection_count['inclusion'],
            current_detection_count['rolled pit'],
            current_detection_count['crease']
        )

        tree.insert("", tk.END, values=new_data)

        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택

            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 2)
            image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # 탐지된 객체 수와 각 객체의 정보를 result_label에 업데이트
        detection_summary_text = f"탐지된 오류 수: {len(final_boxes)}\n\n"
        detection_summary_text += "\n".join(detection_details)  # 탐지된 객체 정보를 줄바꿈으로 나열
        result_label.config(text=detection_summary_text, font=("Helvetica", 16))  # 폰트 크기 16으로 설정

        # 이미지 표시
        show_image_from_array(image)

        # *** 예측된 이미지 저장 ***
        save_image_path = os.path.join(save_path, f"output_{os.path.basename(file_path)}")
        cv2.imwrite(save_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # BGR로 변환하여 저장
        print(f"이미지 저장됨: {save_image_path}")

        # 총합 업데이트
        update_summary()

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")

# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# 결과 레이블 (우측 상단에 고정)
result_label = tk.Label(window, text="")
# result_label.grid(row=0, column=2, padx=10, pady=10, sticky="ne")  # 우측 상단에 고정
result_label.grid(row=0, column=2, padx=(20, 40), pady=(10, 30), sticky="ne")

# 폴더 선택 버튼
folder_button = tk.Button(window, text="폴더 선택 및 분석", command=select_folder)
folder_button.grid(row=4, column=0, pady=10)

# 파일 업로드 버튼 추가
upload_button = tk.Button(window, text="파일 업로드 및 분석", command=upload_and_predict)
upload_button.grid(row=4, column=1, pady=10)

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

window.mainloop()



image 1/1 C:\KDT_SF6\ai\  \img_01_424799300_01133.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 32.9ms
Speed: 2.1ms preprocess, 32.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:\KDT_SF6\비전ai\output_img_01_424799300_01133.jpg

image 1/1 C:\KDT_SF6\ai\  \img_01_424799600_00001.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 51.5ms
Speed: 2.0ms preprocess, 51.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:\KDT_SF6\비전ai\output_img_01_424799600_00001.jpg

image 1/1 C:\KDT_SF6\ai\  \img_01_424799600_00002.jpg: 320x640 1 3_yueyawan, 1 4_shuiban, 35.0ms
Speed: 1.0ms preprocess, 35.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:\KDT_SF6\비전ai\output_img_01_424799600_00002.jpg

image 1/1 C:\KDT_SF6\ai\  \img_01_424825700_00001.jpg: 320x640 1 3_yueyawan, 35.3ms
Speed: 1.0ms preprocess, 35.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:\KDT_SF6\비전ai\output_img_01_424825700_00001.j

In [11]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import os

# YOLO 모델 로드
model = YOLO(r'C:\KDT_SF6\비전ai\best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),   # yaozhe - 빨강
    1: (0, 255, 0),   # waist folding - 초록
    2: (0, 0, 255),   # punching hole - 파랑
    # 나머지 클래스 색상 추가
}

# 이미지 저장 경로 설정
save_path = r'C:\KDT_SF6\비전ai'  # 저장할 폴더 경로

# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# 탐지 결과를 저장할 딕셔너리 (초기값 0)
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (좌측에 이미지 배치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, padx=10, pady=10, sticky="nw")

# 결과 레이블 (사진의 우측에 배치)
result_label = tk.Label(window, text="")
result_label.grid(row=0, column=1, padx=10, pady=10, sticky="nw")  # 이미지의 바로 우측에 배치

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(*summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_list, file_index
    if file_list and file_index < len(file_list):
        file_path = os.path.join(selected_folder, file_list[file_index])
        predict_image(file_path)
        file_index += 1

    # 더 이상 파일이 없으면 종료
    if file_index >= len(file_list):
        result_label.config(text="모든 파일을 처리하였습니다.")
        return

    # 2초마다 이 함수를 다시 호출
    window.after(2000, check_for_new_image)

# 폴더 선택 및 이미지 처리 함수
def select_folder():
    global file_list, file_index, selected_folder
    selected_folder = filedialog.askdirectory()  # 폴더 선택 다이얼로그
    if selected_folder:
        file_list = [f for f in os.listdir(selected_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        file_index = 0
        if file_list:
            check_for_new_image()

# 개별 파일 선택 및 예측 수행 함수
def upload_and_predict():
    file_path = filedialog.askopenfilename(title="이미지 파일 선택", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        predict_image(file_path)

def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 각 클래스별 탐지된 개수를 저장할 임시 딕셔너리
        current_detection_count = {key: 0 for key in custom_names.values()}
        detection_details = []  # 탐지된 객체의 상세 정보를 저장할 리스트

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy
            cls = r.boxes.cls
            conf = r.boxes.conf
            cls_dict = r.names

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box
                x1_int, y1_int = int(x1.item()), int(y1.item())
                x2_int, y2_int = int(x2.item()), int(y2.item())
                cls_name = cls_dict[int(cls_number.item())]
                conf_number = float(conf_value.item())

                # 탐지된 객체 수 업데이트
                current_detection_count[cls_name] += 1
                detection_summary[cls_name] += 1

                # 총합 계산
                total_sums[cls_name] += 1

                # 탐지된 객체 정보 저장
                detection_details.append(f"{cls_name}: ({x1_int}, {y1_int})에서 ({x2_int}, {y2_int}) [확률: {conf_number:.2f}]")

                final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                    "cls_name": cls_name, "conf": conf_number})

        # 테이블에 결과 추가
        new_data = (
            os.path.basename(file_path),
            current_detection_count['waist folding'],
            current_detection_count['punching hole'],
            current_detection_count['welding line'],
            current_detection_count['crescent gap'],
            current_detection_count['water spot'],
            current_detection_count['oil spot'],
            current_detection_count['silk spot'],
            current_detection_count['inclusion'],
            current_detection_count['rolled pit'],
            current_detection_count['crease']
        )

        tree.insert("", tk.END, values=new_data)

        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택

            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 2)
            image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # 탐지된 객체 수와 각 객체의 정보를 result_label에 업데이트
        detection_summary_text = f"탐지된 오류 수: {len(final_boxes)}\n\n"
        detection_summary_text += "\n".join(detection_details)  # 탐지된 객체 정보를 줄바꿈으로 나열
        result_label.config(text=detection_summary_text, font=("Helvetica", 16))  # 폰트 크기 16으로 설정

        # 이미지 표시
        show_image_from_array(image)

        # *** 예측된 이미지 저장 ***
        save_image_path = os.path.join(save_path, f"output_{os.path.basename(file_path)}")
        cv2.imwrite(save_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # BGR로 변환하여 저장
        print(f"이미지 저장됨: {save_image_path}")

        # 총합 업데이트
        update_summary()

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")

# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# 폴더 선택 버튼
folder_button = tk.Button(window, text="폴더 선택 및 분석", command=select_folder)
folder_button.grid(row=4, column=0, pady=10)

# 파일 업로드 버튼 추가
upload_button = tk.Button(window, text="파일 업로드 및 분석", command=upload_and_predict)
upload_button.grid(row=4, column=1, pady=10)

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

window.mainloop()



image 1/1 C:\KDT_SF6\ai\  \img_01_424799300_01133.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 43.0ms
Speed: 2.0ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:\KDT_SF6\비전ai\output_img_01_424799300_01133.jpg

image 1/1 C:\KDT_SF6\ai\  \img_01_424799600_00001.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 28.9ms
Speed: 1.0ms preprocess, 28.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:\KDT_SF6\비전ai\output_img_01_424799600_00001.jpg

image 1/1 C:\KDT_SF6\ai\  \img_01_424799600_00002.jpg: 320x640 1 3_yueyawan, 1 4_shuiban, 32.9ms
Speed: 1.0ms preprocess, 32.9ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:\KDT_SF6\비전ai\output_img_01_424799600_00002.jpg

image 1/1 C:\KDT_SF6\ai\  \img_01_424825700_00001.jpg: 320x640 1 3_yueyawan, 34.9ms
Speed: 1.0ms preprocess, 34.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:\KDT_SF6\비전ai\output_img_01_424825700_00001.j